In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt



In [8]:
# tensor 就是 PyTorch 的“矩阵/向量容器”
a = torch.tensor([1, 2, 3], dtype=torch.float32)
b = torch.zeros((2, 3))    # 2x3 矩阵，全 0
c = torch.ones((2, 2))     # 全 1
d = torch.rand((2, 2))     # 0~1 均匀分布
e = torch.randn((2, 2))    # 正态分布

# 看看形状和类型
print(a, a.shape, a.dtype)
print(b)
print(c)
print(d)
print(e)


tensor([1., 2., 3.]) torch.Size([3]) torch.float32
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[0.5412, 0.9746],
        [0.4062, 0.1236]])
tensor([[-0.1903, -0.7232],
        [-0.1780,  0.5354]])


In [6]:
print(a, a.shape, a.dtype)
print(b, b.shape)
print(c)
print(d)
print(e)

tensor([1., 2., 3.]) torch.Size([3]) torch.float32
tensor([[0., 0., 0.],
        [0., 0., 0.]]) torch.Size([2, 3])
tensor([[1., 1.],
        [1., 1.]])
tensor([[0.6043, 0.3888],
        [0.4953, 0.4694]])
tensor([[ 1.4588,  1.3292],
        [-1.6625, -0.1007]])



tensor vs NumPy array
  - tensor 是 PyTorch 的基本数据结构，可以直接在 GPU 上运行；  
  - NumPy array 只能在 CPU 上运行，不能自动求导。  
rand vs randn
  - `rand` 生成 [0,1) 区间的均匀分布随机数；  
  - `randn` 生成均值=0、方差=1 的正态分布随机数

用 PyTorch 的 autograd 机制，计算一个简单的梯度

In [12]:
# --- Cell (Code) ---
a = torch.tensor([2.0, 3.0], requires_grad=True)  # 参数 a
b = torch.tensor([4.0, 5.0])                      # 常量 b
c = (a * b).sum()                                 # 损失函数 c = 2*4 + 3*5 = 23
c.backward()                                      # 自动计算梯度

print("梯度 a.grad:", a.grad)

梯度 a.grad: tensor([4., 5.])


在神经网络里，这样的自动求导有什么用？
  - 神经网络的损失函数 L 对所有参数 θ 自动求导 → 得到梯度 → 用梯度下降更新参数。  
  - 避免手写复杂的链式法则，大大简化训练过程。  

写一个最小模型，用梯度下降拟合 y=2x+3

In [13]:
# 数据：y = 2x + 3
x = torch.linspace(-5, 5, 20).unsqueeze(1)  # shape: (20, 1) 20 行、每行 1 个特征”的小批量数据
y = 2 * x + 3

# 参数初始化随机，让他拟合实际值
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

learning_rate = 0.01

for epoch in range(200):
    # 前向传播
    y_pred = w * x + b
    loss = ((y_pred - y) ** 2).mean()  # MSE 均方误差
    
    # 反向传播
    loss.backward()
    
    # 参数更新 (手动 SGD)
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
    
    # 梯度清零
    # PyTorch 的梯度是累积的（accumulate），如果不清零，下一轮会把新旧梯度加在一起
    w.grad.zero_()
    b.grad.zero_()
    
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}: loss={loss.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")



Epoch 50: loss=1.3011, w=2.0000, b=1.8821
Epoch 100: loss=0.1726, w=2.0000, b=2.5929
Epoch 150: loss=0.0229, w=2.0000, b=2.8518
Epoch 200: loss=0.0030, w=2.0000, b=2.9460


为什么要按照 Forward → Loss → Backward → Update 的顺序来做：

Forward（前向传播）：输入数据经过模型计算输出（预测值）。

Loss（损失计算）：比较预测值和真实值，算出误差。

Backward（反向传播）：通过链式法则计算损失对参数的梯度。

Update（参数更新）：利用梯度下降法调整参数，让下一次预测更接近真实值。

整个过程像一个闭环，重复很多次，模型就越来越“聪明”。